In [0]:
import pandas as pd
import numpy as np
import pickle
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [29]:

train_data = pd.read_csv(r'/content/drive/My Drive/train_data_v3.csv')
df_train = pd.DataFrame(train_data,columns=['Reported Verbatim','Lowest Level Term (LLT)'])
df_train.head()

,Reported Verbatim,Lowest Level Term (LLT)
0,Event Verbatim: dries me out like no other pro...,Application site reaction
1,Event Verbatim: Fabric part of bandage cause r...,Application site rash
2,"Event Verbatim: I have a complaint to make, ab...",Application site excoriation
3,Event Verbatim: I used a Tough Strip on my bru...,Intentional device misuse
4,Event Verbatim: I began using 3 of the Absolut...,Application site redness


In [30]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24126 entries, 0 to 24125
Data columns (total 2 columns):
Reported Verbatim          24126 non-null object
Lowest Level Term (LLT)    24126 non-null object
dtypes: object(2)
memory usage: 377.0+ KB


In [31]:
test_data = pd.read_csv('/content/drive/My Drive/test_data_v3.csv')
df_test = pd.DataFrame(test_data,columns=['Reported Verbatim','Lowest Level Term (LLT)'])
df_test.head()

,Reported Verbatim,Lowest Level Term (LLT)
0,Event Verbatim: Health Canada AER 000063104: A...,Abdominal discomfort
1,Event Verbatim: Have been using this product f...,Abnormal hair growth
2,Event Verbatim: I purchased the NTG T/SAL MAX ...,Abnormal sensation in eye
3,Event Verbatim: I used one of your flexible Ba...,Abrasions
4,Event Verbatim: I used this and did not know t...,Accidental contact of product with eye


In [32]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10601 entries, 0 to 10600
Data columns (total 2 columns):
Reported Verbatim          10601 non-null object
Lowest Level Term (LLT)    10601 non-null object
dtypes: object(2)
memory usage: 165.7+ KB


In [33]:
df_train['Lowest Level Term (LLT)'].value_counts()

Application site reaction                   1725
Lack of drug effect                         1620
Unexpected therapeutic effect               1562
Off label use                               1487
Application site rash                       1190
Product quality issue                        939
Drug effect incomplete                       791
Application site redness                     751
Application site dryness                     688
Application site acne                        580
Application site burning                     534
Application site itching                     531
Application site excoriation                 506
Application site swelling                    458
Application site papules                     416
Device adhesion issue                        415
Application site pain                        364
Application site discoloration               332
Product physical consistency issue           290
Application site irritation                  262
Wrong technique in p

In [34]:
df_test['Lowest Level Term (LLT)'].value_counts()

Application site reaction                   740
Lack of drug effect                         696
Unexpected therapeutic effect               671
Off label use                               637
Application site rash                       509
Product quality issue                       402
Drug effect incomplete                      339
Application site redness                    323
Application site dryness                    296
Application site acne                       249
Application site burning                    230
Application site itching                    229
Application site excoriation                217
Application site swelling                   197
Application site papules                    180
Device adhesion issue                       179
Application site pain                       156
Application site discoloration              143
Product physical consistency issue          126
Application site irritation                 114
Wrong technique in product usage process

In [0]:
#Text Preprocessing

def print_plot(index):
    example = df_test[df_test.index == index][['Reported Verbatim', 'Lowest Level Term (LLT)']].values[0]
    if len(example) > 0:
        print(example[0])
        print('LLT:', example[1])


In [36]:
print_plot(10)

Event Verbatim: My one year old grand daughter had some of this sunscreen in her mouth.   Verbatim: My one year old grand daughter had some of this sunscreen in her mouth.
LLT: Accidental drug intake by child


In [37]:
print_plot(20)

Event Verbatim: I was just talking to someone who gave me a reference number. He didn't get my last name. I called because my son had gotten some baby powder on his tongue. I was told to call Poison Control and then call you back.  Summary: On October 12, 2017, a mother called to report her 2 year old, 36lb, Caucasian son (the patient) had gotten a hold of some Johnson's Baby Powder and accidentally shook some onto his tongue. Mother states that she contacted Poison Control and was assured patient would be alright, at which time mother called to report to us. Mother states that she was traveling in the car with the patient, when the patient grabbed the Baby Powder from the carrying bag and sprinkled about a dime sized amount onto his tongue. Mother indicates that she will continue to use Baby Powder on patient. Mother reports that patient is fine and shows no signs of illness.   Verbatim: I was just talking to someone who gave me a reference number. He didn't get my last name. I called

In [38]:
import nltk
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.corpus import stopwords


df_train = df_train.reset_index(drop=True)
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text. substitute the matched string in REPLACE_BY_SPACE_RE with space.
    text = BAD_SYMBOLS_RE.sub('', text) # remove symbols which are in BAD_SYMBOLS_RE from text. substitute the matched string in BAD_SYMBOLS_RE with nothing. 
    text = text.replace('Verbatim:', '')
#    text = re.sub(r'\W+', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # remove stopwors from text
    return text
df_train['Reported Verbatim'] = df_train['Reported Verbatim'].apply(clean_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [0]:
df_train['Reported Verbatim'] = df_train['Reported Verbatim'].str.replace('\d+', '')

In [0]:
#Tokenizing the Reported Verbatim of test and train dataframes
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=10000, lower=True)
tokenizer.fit_on_texts(df_train['Reported Verbatim'])
sequences1 = tokenizer.texts_to_sequences(df_train['Reported Verbatim'])
x_train = pad_sequences(sequences1, maxlen=180)


tokenizer = Tokenizer(num_words=10000, lower = True)
tokenizer.fit_on_texts(df_test['Reported Verbatim'])
sequences2 = tokenizer.texts_to_sequences(df_test['Reported Verbatim'])
x_test = pad_sequences(sequences2, maxlen=180)



In [41]:
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_train['Lowest Level Term (LLT)'])
y_train = multilabel_binarizer.transform(df_train['Lowest Level Term (LLT)'])
print(len(y_train))

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(df_test['Lowest Level Term (LLT)'])
y_test = multilabel_binarizer.transform(df_test['Lowest Level Term (LLT)'])
print(len(y_test))

24126
10601


In [42]:
from keras.layers import Activation, Dense, Dropout
from sklearn.preprocessing import LabelBinarizer
import sklearn.datasets as skds
from pathlib import Path
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

model_lstm = Sequential()
model_lstm.add(Embedding(10000, 180, input_length=180))
model_lstm.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model_lstm.add(Dense(51, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model_lstm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 180, 180)          1800000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               112400    
_________________________________________________________________
dense_3 (Dense)              (None, 51)                5151      
Total params: 1,917,551
Trainable params: 1,917,551
Non-trainable params: 0
_________________________________________________________________


In [43]:
model_lstm.fit(x_train,y_train, validation_split=0.4, epochs=4)

Train on 14475 samples, validate on 9651 samples
Epoch 1/4
14475/14475 [==============================] - 202s 14ms/step - loss: 0.2859 - acc: 0.8836 - val_loss: 0.2686 - val_acc: 0.8872
Epoch 2/4
14475/14475 [==============================] - 201s 14ms/step - loss: 0.2610 - acc: 0.8921 - val_loss: 0.2509 - val_acc: 0.8970
Epoch 3/4
14475/14475 [==============================] - 201s 14ms/step - loss: 0.2433 - acc: 0.9005 - val_loss: 0.2465 - val_acc: 0.8989
Epoch 4/4
14475/14475 [==============================] - 201s 14ms/step - loss: 0.2336 - acc: 0.9046 - val_loss: 0.2457 - val_acc: 0.9000


In [45]:
score = model_lstm.evaluate(x_test, y_test,
                       batch_size=1000, verbose=1)
 
print('Test accuracy:', score[1])
print('Accuracy:',score[1]*100)
 
text_labels = multilabel_binarizer.classes_


10601/10601 [==============================] - 19s 2ms/step
Test accuracy: 0.8857957305756169
Accuracy: 88.57957305756169


In [49]:
#Calculating the test accuracy for each LLT
categories = list(test_data.columns.values)

 
for category in categories:
    print('**Processing {} comments...**'.format(category))
    
    # Training lstm  model on train data
   # model_lstm.fit(x_train,y_train, validation_split=0.4, epochs=3)
    score = model_lstm.evaluate(x_test, y_test,batch_size=1000, verbose=1)
    
    # calculating test accuracy
    print('Test accuracy:', score[])
    print("\n")

**Processing Case Tracking_No comments...**
10601/10601 [==============================] - 19s 2ms/step


TypeError: ignored

In [27]:
new_complaint = ['Event Verbatim: On 05/25/2017 a female patient reported by email: I used this product twice when I went for a leisurely bike ride. Both times it completely *blocked the pores* on my face where I wound up with *HEAT STROKE*! The first time it happened, I never even thought it could be from your sunscreen, until the same thing happened the next time! It wasnt even too hot either day....70 degrees with hardly any humidity. My face *turned bright red and hot!* I got a *splitting headache* so bad it made me *nauseous!* I had *cramps* in both sides of my abdomen near my hips. I didnt sweat but it was like a had a plastic bag over my head! And my heart was racing! I truly felt like I was going to die!!! It wasnt until I was able to completely wash it off my face that I began feeling like I wasnt being suffocated! I had to cool down with ice packs on my forehead and neck. I was NOT dehydrated...I drank lots of fluids and my urine was a normal color. It was HEAT stroke!!! I think you need to retest this product before someone does die! Honestly! \\\ No further information provided.   Verbatim: Both times it completely blocked the pores on my face where I wound up with HEAT STROKE!...My face turned bright red and hot! I got a splitting headache so bad it made me nauseous! I had cramps in both sides of my abdomen near my hips.(SGN)  Performed 15 Day Follow-Up For: AE Please Call Us/Lot Request â€“ Sent Email']
seq = tokenizer.texts_to_sequences(new_complaint)
padded = pad_sequences(seq, maxlen=180)
pred = model_lstm.predict(padded)
labels = list(test_data.columns.values)
print(pred, labels[np.argmax(pred)])

[[9.78498816e-01 2.52366066e-04 1.71631575e-04 7.75754452e-05
  4.60849911e-01 4.03013825e-03 5.32960892e-03 9.79430079e-02
  1.08859837e-02 1.84115767e-03 2.27689743e-04 9.23237205e-03
  1.36463046e-02 3.64980400e-02 7.40706921e-04 2.83235312e-03
  2.79667079e-02 7.89471567e-02 2.67925858e-03 1.12852454e-02
  3.49017978e-03 1.30518079e-02 5.19666076e-03 2.05186307e-02
  1.12742186e-04 9.25609350e-01 1.11171544e-01 9.37593818e-01
  5.36524057e-01 9.80954289e-01 2.47384042e-01 2.81259000e-01
  1.53690457e-01 9.05859113e-01 7.96794891e-04 7.35816061e-02
  7.41346359e-01 1.05748981e-01 8.05602193e-01 8.81126761e-01
  6.04735374e-01 5.43763041e-02 7.14447141e-01 7.90029883e-01
  9.03847337e-01 5.07569015e-01 9.98073816e-02 1.50029361e-02
  9.09642577e-02 1.34849027e-01 3.06235976e-03]] Application site dryness
